In this notebook we try to calculate 1 subjects AHI and RDI index from the XML annotation file, and see if it matches the calculated indexes given in the CSV file.

The reason why is because of known issues with oxygen desaturation (SpO2) linking/missingness.

In [25]:
import numpy as np
import os
import pandas as pd
import csv
import xml.etree.ElementTree

In [2]:
basedir = '/Users/morten/Dropbox-personal/Dropbox/master-thesis/shhs/'
fname_xml = "%s/polysomnography/nssr/shhs2/shhs2-200081-nsrr.xml" % basedir
fname_csv = "%s/shhs2-dataset-0.13.0.csv" % basedir

Function to read the the xml annotation file (not pretty)

In [3]:
def read_xml(xml_fp):
    e = xml.etree.ElementTree.parse(xml_fp).getroot()
    scored_events = e.find("ScoredEvents")

    col = ['EventType','EventConcept','Start','Duration','SignalLocation','SpO2Nadir','SpO2Baseline']
    df_scored_events = pd.DataFrame(columns=col)
    df_sleep_stages = pd.DataFrame(columns=['EventType','EventConcept','Start','Duration'])
    duration = 0
    for event in scored_events:
        if event.find("EventConcept").text == "Recording Start Time":
            duration = float(event.find("Duration").text)
        if event.find("EventType").text:
            if event.find("EventType").text == "Stages|Stages":
                df_sleep_stages = df_sleep_stages.append(pd.Series([event.find("EventType").text, 
                                                                    event.find("EventConcept").text, 
                                                                    event.find("Start").text, 
                                                                    event.find("Duration").text], 
                                                                   index=['EventType','EventConcept','Start','Duration']), 
                                                         ignore_index=True)
            elif event.find("EventConcept").text == 'SpO2 desaturation|SpO2 desaturation':
                 df_scored_events = df_scored_events.append(pd.Series([event.find("EventType").text, 
                                                                      event.find("EventConcept").text, 
                                                                      event.find("Start").text, 
                                                                      event.find("Duration").text, 
                                                                      event.find("SignalLocation").text,
                                                                      event.find("SpO2Nadir").text,
                                                                      event.find("SpO2Baseline").text],
                                                                     index=col), 
                                                           ignore_index=True)
                
            else:
                df_scored_events = df_scored_events.append(pd.Series([event.find("EventType").text, 
                                                                      event.find("EventConcept").text, 
                                                                      event.find("Start").text, 
                                                                      event.find("Duration").text, 
                                                                      event.find("SignalLocation").text,
                                                                      None, None],
                                                                     index=col), 
                                                           ignore_index=True)
    
    
    return [df_scored_events, df_sleep_stages, duration]

In [4]:
df_scored_events, df_sleep_stages, duration = read_xml(fname_xml)
print("Recording duration: %d sec." % duration)

Recording duration: 42630 sec.


In [5]:
df_csv = pd.read_csv(fname_csv, encoding = "ISO-8859-1")

In [6]:
df_csv.head(5)

,nsrrid,pptid,ecgdate,lvh3_1,lvh3_3,st4_1_3,st5_1_3,lvhst,ventrate,qrs,...,QuAir,QuChest,QuAbdo,QuOxim,ligh,latreliable,Posn,gender,race,age_s1
0,200077,77,1915.0,0.0,0.0,0.0,0.0,0.0,73.0,18.0,...,5.0,5.0,5.0,5.0,1.0,1.0,1.0,1,1,41
1,200078,78,1920.0,0.0,0.0,0.0,0.0,0.0,61.0,-26.0,...,5.0,3.0,5.0,5.0,1.0,1.0,1.0,1,1,54
2,200079,79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,4.0,5.0,5.0,1.0,1.0,1.0,2,3,56
3,200080,80,1911.0,0.0,0.0,0.0,0.0,0.0,79.0,53.0,...,5.0,5.0,5.0,5.0,1.0,1.0,1.0,1,1,54
4,200081,81,1917.0,0.0,0.0,0.0,1.0,0.0,75.0,81.0,...,4.0,5.0,5.0,5.0,1.0,1.0,1.0,2,1,40


In [7]:
variables = df_csv[df_csv['nsrrid'] == 200081]

In [8]:
print("Minutes of sleep: %d" % variables['slpprdp']) # 433 minutes of sleep for subject: shhs2-200081-nsrr.xml
sleep_time = variables['slpprdp']
print("Central Apnea Index %0.2f" % variables['cai0p']) #  0.69 Central Apnea Index - numer of [central apneas] per hour of sleep
print("RDI: %0.2f" % variables['rdi0p']) # 8.59 Overall Respiratory Disturbance Index (RDI) all oxygen desaturations)
print("AHI: %0.2f" % variables['ahi_a0h3']) # 2.63 Apnea-Hypopnea Index (AHI) >= 3% - number of [all apneas] and [hypopneas with >= 3% oxygen desaturation] per hour of sleep

Minutes of sleep: 433
Central Apnea Index 0.69
RDI: 8.59
AHI: 2.63


Calculation formel for AHI: https://sleepdata.org/datasets/shhs/variables/ahi_a0h3

The procedure is the same for all variables. What differs is how/what we calculate as events:

Index (events pr. hour) = 60 * events / minutes of sleep

However we have to remove events scored in a Wake epoch. 
But if a scored event streches over multiple epochs and one of them is a Sleep Stage it will not be removed.

In [9]:
df = df_scored_events
central_apnea_events = df[df["EventConcept"] == "Central apnea|Central Apnea"]
events = len(central_apnea_events)
print("Number of central apnea events: %d" % events)
print("Central Apnea Index (CAI) calculated: %0.2f, expected: %0.2f" % ((60*events/sleep_time), variables['cai0p']))

Number of central apnea events: 5
Central Apnea Index (CAI) calculated: 0.69, expected: 0.69


We got the right index without filtering wake events

In [10]:
rdi_events = df[df["EventConcept"] == "Hypopnea|Hypopnea"]
rdi_events = rdi_events.append(central_apnea_events)
events = len(rdi_events)
print("Number of RDI events: %d" % events)
print("RDI calculated: %0.2f, expected: %0.2f" % ((60*events/sleep_time), variables['rdi0p']))

Number of RDI events: 63
RDI calculated: 8.73, expected: 8.59


In this case we have to remove exactly 1 event scored in a Wake epoch.
Lots of ways to do this and this is probably not the best nor prettiest way

In [11]:
wake_stages = df_sleep_stages[df_sleep_stages["EventConcept"] == "Wake|0"]

In [12]:
rdi_events
remove_indexes = []
index = 0
for start, dur in zip(pd.to_numeric(rdi_events["Start"]), pd.to_numeric(rdi_events["Duration"])):
    for w_start, w_dur in zip(pd.to_numeric(wake_stages["Start"]), pd.to_numeric(wake_stages["Duration"])):
        if start >= w_start and start < w_start+w_dur:
            remove_indexes.append(index)     
    index += 1

In [13]:
print("rdi_events before removal: %d" % len(rdi_events))

for i in remove_indexes:
    rdi_events = rdi_events.drop(rdi_events.index[i])

print("rdi_events after removal: %d" % len(rdi_events))

rdi_events before removal: 63
rdi_events after removal: 62


In [14]:
events = len(rdi_events)
print("Number of RDI events: %d" % events)
print("RDI calculated: %0.2f, expected: %0.2f" % ((60*events/sleep_time), variables['rdi0p']))

Number of RDI events: 62
RDI calculated: 8.59, expected: 8.59


Overall Respiratory Disturbance Index (RDI) uses oxygen desaturation at all levels. 
But for AHI we need to filter the Hypopneas using the the SpO2 desaturation to look for events only with oxygen saturation >= 3 %.

And here comes the problem:

In [15]:
desat = df[df["EventConcept"] == "SpO2 desaturation|SpO2 desaturation"]
hypopnea = df[df["EventConcept"] == "Hypopnea|Hypopnea"]

print("SpO2 desaturation events: %d " % len(desat))
print("Hypopnea events: %d" % len(hypopnea))

SpO2 desaturation events: 42 
Hypopnea events: 58


Meaning we are missing 16 desaturation events for the corresponding hypopnea events.

In [16]:
desaturation_above_threshold = 0
for nadir, baseline in zip(pd.to_numeric(desat['SpO2Nadir']),pd.to_numeric(desat['SpO2Baseline'])):
    if baseline-nadir >= 3:
        desaturation_above_threshold +=1

In [17]:
print("Desaturation above threshold: %d" % desaturation_above_threshold)

Desaturation above threshold: 9


We can add the central_apnea_events because we don't have to use look at the desaturation for them as pr. definition of AHI. 

In [18]:
events = desaturation_above_threshold + len(central_apnea_events)
print("Number of AHI events: %d" % events)
print("AHI calculated: %0.2f, expected: %0.2f" % ((60*events/sleep_time), variables['ahi_a0h3']))

Number of AHI events: 14
AHI calculated: 1.94, expected: 2.63


Meaning we are missing 5 Hypopnea events to get 19 total events which is number needed to match the calculated AHI:

In [19]:
print("Number of AHI events: %d" % 19)
print("AHI calculated: %0.2f, expected: %0.2f" % ((60*19/sleep_time), variables['ahi_a0h3']))

Number of AHI events: 19
AHI calculated: 2.63, expected: 2.63
